In [1]:
# import numpy as np
# import scipy.linalg as la

# def alternating_minimization(Xs, Ys, Us, Zs, 
#                              nus: np.array = np.array(1), 
#                              reg: np.array = np.array(0), 
#                              symmetric_A: bool = True,
#                              max_iter = 100, tol = 1e-8):
#     '''
#     Routine which infers A, B in Z = XAY + BU with optional
#     symmetry constraint on A.  Supports scalar regularization
#     and affine parametric dependence in A,B.
#     '''
#     _, Us, _, _, _ = _fix_input_dims_(Xs, Us, Zs, nus)
#     Xs, Ys, Zs, nus, case = _fix_input_dims_(Xs, Ys, Zs, nus)

#     if symmetric_A:
#         func_A = lambda nus, Xs, Ys, Zs: infer_Tbar_with_symmetry(nus, Xs, Ys,
#                                                                   Zs, reg=reg, symmetric=True)
#     else:
#         func_A = lambda nus, Xs, Ys, Zs: infer_Tbar_with_lstsq(nus, Ys, Zs, reg=reg)
    
#     func_B = infer_Tbar_with_lstsq
    
#     A = func_A(nus, Xs, Ys, Zs)
#     B = func_B(nus, Us, Zs, reg=reg)
    
#     for i in range(max_iter):
#         # Minimize A given B
#         rhs_A = Zs - np.einsum('ibp,ps,bas->ias', B, nus, Us)
#         A_new = func_A(nus, Xs, Ys, rhs_A)
        
#         # Minimize B given A
#         rhs_B = Zs - np.einsum('cds,dep,ps,eas->cas', Xs, A_new, nus, Ys)
#         B_new = func_B(nus, Us, rhs_B, reg=reg)

#         # Check for convergence
#         if np.linalg.norm(A_new - A) < tol and np.linalg.norm(B_new - B) < tol:
#             print(f"Converged in {i+1} iterations.")
#             break
        
#         A = A_new
#         B = B_new
    
#     return (A, B) if case==2 else (A.squeeze(), B.squeeze())


# def symmetric_OpInf(Xs, Ys, Zs, nus: np.array = np.array(1),
#                     reg: np.array = np.array(0), symmetric: bool = True):
#     '''
#     Routine which infers a symmetric/antisymmetric operator
#     '''
#     Xs, Ys, Zs, nus, case = _fix_input_dims_(Xs, Ys, Zs, nus)
#     T = infer_Tbar_with_symmetry(nus, Xs, Ys, Zs, reg, symmetric)

#     return T if case==2 else T.squeeze()


# def unstructured_OpInf(Ys, Zs, nus: np.array = np.array(1), 
#                        reg: np.array = np.array(0)):
#     '''
#     Routine which infers an unstructured operator 
#     '''
#     _, Ys, Zs, nus, case = _fix_input_dims_(Ys, Ys, Zs, nus)
#     T = infer_Tbar_with_lstsq(nus, Ys, Zs, reg)

#     return T if case==2 else T.squeeze()


# def infer_Tbar_with_lstsq(nus, Ys, Zs, reg: np.array = np.array(0)):
#     '''
#     Infers an unstructured tensor operator.
#     '''
#     p, Ns = nus.shape
#     b, Nt, Ns = Ys.shape
#     r, Nt, Ns = Zs.shape 

#     # Reshape regularization into rp x rp matrix
#     if len(reg.shape) == 0:
#         reg = reg * np.eye(b*p)
#     elif len(reg.shape) == 1:
#         reg = np.diag(reg)

#     K  = np.einsum("ias,xs->iaxs", Ys, nus)
#     Dt = K.transpose((0, 2, 1, 3)).reshape((b*p, Nt*Ns), order="F")
#     R  = Zs.reshape((r, Nt*Ns), order="F")

#     if reg.sum() != 0.:
#         Dt = np.concatenate([Dt, reg], axis=1)
#         R  = np.concatenate([R, np.zeros((r, b*p))], axis=1)

#     Obar = np.linalg.lstsq(Dt.T, R.T)[0].T
#     return Obar.reshape((r, b, p), order="F")


# def infer_Tbar_with_symmetry(nus, Xs, Ys, Zs, reg: np.array = np.array(0),
#                              symmetric: bool = True):    
#     '''
#     Infers a tensor operator with symmetry or antisymmetry
#     '''
#     p, Ns = nus.shape
#     r, Nt, Ns = Ys.shape  # or Zs.shape

#     # Reshape regularization into r^2p x r^2p matrix
#     if len(reg.shape) == 0:
#         reg = reg * np.eye(r*r*p)
#     elif len(reg.shape) == 1:
#         reg = np.diag(reg)

#     XsTXs = np.einsum("kis,kjs->ijs", Xs, Xs)
#     YsYsT = np.einsum("iks,jks->ijs", Ys, Ys)
#     nusnusT = np.einsum("xs,ys->xys", nus, nus)

#     Btsr = np.einsum("xys,ijs,kls->xyijkl", nusnusT, XsTXs, YsYsT)
#     Btsr += Btsr.transpose(0, 1, 4, 5, 2, 3)  # tensorized Kronecker sum
#     Bhat = Btsr.transpose(0, 2, 4, 1, 3, 5).reshape((r*r*p, r*r*p), order="C")

#     if reg.sum() != 0.:
#         Bhat += reg

#     Ctsr = np.einsum("kis,kas,jas,xs->ijx", Xs, Zs, Ys, nus)
#     Ctsr += (1 if symmetric else -1) * Ctsr.transpose(1, 0, 2)
#     Chat = Ctsr.flatten(order="F")

#     vecT = la.solve(Bhat, Chat, assume_a="sym")
#     return vecT.reshape((r, r, p), order="F")


# def _fix_input_dims_(Xs, Ys, Zs, nus):
#     '''
#     Helper function to fix dimensions of inputs to p-OpInf
#     '''
#     if len(nus.shape) == 0:
#         # No parametric dependence
#         # Expected nus = np.array(1)
#         nus = nus[None,None]
#         case = 0
        
#         if len(Ys.shape) == 2:
#             # Only samples in time dimension
#             # Expected Ys,Zs have dim 2
#             Xs = Xs[:,:,None]
#             Ys = Ys[:,:,None]
#             Zs = Zs[:,:,None]

#     elif len(nus.shape) == 1:
#         # Single parametric dependence
#         # Expected nus = np.ones(Ns) or Ns parameter samples
#         nus = nus[None,:]
#         case = 1

#     else:
#         case = 2

#     p, Ns = nus.shape
#     b, Nt, Ns = Ys.shape 
#     r, Nt, Ns = Zs.shape 

#     if len(Xs.shape) == 2:
#         # No sample dependence in X matrix
#         Xs = Xs[:,:,None] * np.ones([r,r,Ns])

#     return Xs, Ys, Zs, nus, case

In [2]:
import opinf_utils_anthony as ou
import numpy as np

r = 10
Nt = 15
Ns = 25
p = 4

#### Testing on antisymmetric tensor operator (case 2)

nus = np.random.randn(p,Ns)
# Xs = np.eye(r,r)[:,:,None] * np.ones([r,r,Ns])
Xs = np.eye(r,r)
Ys = np.random.randn(r,Nt,Ns)

T_fac = np.random.randn(r,r,p)
T_real = T_fac - T_fac.transpose(1,0,2)

Zs = np.einsum('ab,bcx,xs,cms->ams', Xs, T_real, nus, Ys)

T_lrnd = ou.symmetric_OpInf(Xs, Ys, Zs, nus, symmetric=False)

print(np.allclose(T_real, T_lrnd))


#### Testing on symmetric tensor with one parameter (case 1)

nus = np.random.randn(Ns)
# Xs = np.eye(r,r)[:,:,None] * np.ones([r,r,Ns])
Xs = np.eye(r,r)
Ys = np.random.randn(r,Nt,Ns)

T_fac = np.random.randn(r,r)
T_real = T_fac + T_fac.transpose(1,0)

Zs = np.einsum('ab,bc,s,cms->ams', Xs, T_real, nus, Ys)

T_lrnd = ou.symmetric_OpInf(Xs, Ys, Zs, nus, symmetric=True)

np.allclose(T_real, T_lrnd)

print(np.allclose(T_real, T_lrnd))


#### Testing on symmetric matrix operator (case 0)

Xs = np.eye(r,r)
Ys = np.random.randn(r,Nt)

T_fac = np.random.randn(r,r)
T_real = T_fac + T_fac.T

Zs = Xs @ T_real @ Ys

T_lrnd = ou.symmetric_OpInf(Xs, Ys, Zs, symmetric=True)

print(np.allclose(T_real, T_lrnd))


#### Testing on unstructured tensor operator (case 2)

nus = np.random.randn(p,Ns)
# Xs = np.eye(r,r)[:,:,None] * np.ones([r,r,Ns])
Xs = np.eye(r,r)
Ys = np.random.randn(r,Nt,Ns)

T_real = np.random.randn(r,r,p)

Zs = np.einsum('ab,bcx,xs,cms->ams', Xs, T_real, nus, Ys)

T_lrnd = ou.unstructured_OpInf(Ys, Zs, nus)

print(np.allclose(T_real, T_lrnd))


#### Testing on unstructured tensor with one parameter (case 1)

nus = np.random.randn(Ns)
# Xs = np.eye(r,r)[:,:,None] * np.ones([r,r,Ns])
Xs = np.eye(r,r)
Ys = np.random.randn(r,Nt,Ns)

T_real = np.random.randn(r,r)

Zs = np.einsum('ab,bc,s,cms->ams', Xs, T_real, nus, Ys)

T_lrnd = ou.unstructured_OpInf(Ys, Zs, nus)

np.allclose(T_real, T_lrnd)

print(np.allclose(T_real, T_lrnd))


#### Testing on unstructured matrix operator (case 0)

Xs = np.eye(r,r)
Ys = np.random.randn(r,Nt)

T_real = np.random.randn(r,r)

Zs = Xs @ T_real @ Ys

T_lrnd = ou.unstructured_OpInf(Ys, Zs)

print(np.allclose(T_real, T_lrnd))


#### Testing unstructured regularization (cases 2 and 0)

nus = np.random.randn(p,Ns)
# reg = np.array([1e-5 for i in range(r*p)])
reg = np.array(1e-5)
# Xs = np.eye(r,r)[:,:,None] * np.ones([r,r,Ns])
Xs = np.eye(r,r)
Ys = np.random.randn(r,Nt,Ns)

T_real = np.random.randn(r,r,p)

Zs = np.einsum('ab,bcx,xs,cms->ams', Xs, T_real, nus, Ys)

T_lrnd = ou.unstructured_OpInf(Ys, Zs, nus, reg)

print(np.allclose(T_real, T_lrnd))

reg = np.array([1e-5 for i in range(r)])
Xs = np.eye(r,r)
Ys = np.random.randn(r,Nt)

T_real = np.random.randn(r,r)

Zs = Xs @ T_real @ Ys

T_lrnd = ou.unstructured_OpInf(Ys, Zs, reg=reg)

print(np.allclose(T_real, T_lrnd))


#### Testing structured regularization (cases 2 and 0)

nus = np.random.randn(p,Ns)
# Xs = np.eye(r,r)[:,:,None] * np.ones([r,r,Ns])
# reg = np.array([1e-5 for i in range(r*r*p)])
reg = np.array(1e-8)
Xs = np.eye(r,r)
Ys = np.random.randn(r,Nt,Ns)

T_fac = np.random.randn(r,r,p)
T_real = T_fac - T_fac.transpose(1,0,2)

Zs = np.einsum('ab,bcx,xs,cms->ams', Xs, T_real, nus, Ys)

T_lrnd = ou.symmetric_OpInf(Xs, Ys, Zs, nus, reg=reg, symmetric=False)

print(np.allclose(T_real, T_lrnd))

Xs = np.eye(r,r)
Ys = np.random.randn(r,Nt)

T_fac = np.random.randn(r,r)
T_real = T_fac + T_fac.T

Zs = Xs @ T_real @ Ys

T_lrnd = ou.symmetric_OpInf(Xs, Ys, Zs, reg=reg, symmetric=True)

print(np.allclose(T_real, T_lrnd))


True
True
True
True
True
True
True
True
True
True


/Users/adgrube/Desktop/Code/reentry_rom_mfuq/mfmc_prototype/opinf_utils_anthony.py:92: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Obar = np.linalg.lstsq(Dt.T, R.T)[0].T


In [3]:
#### Test of alt min with no parametric dependence

r = 10
b = 43
Nt = 150

Xs = np.eye(r)
Ys = np.random.randn(r,Nt)
Us = np.random.randn(b,Nt)

T_fac = np.random.randn(r,r)
T_real = T_fac + T_fac.T
B_real = np.random.randn(r,b)

Zs = Xs @ T_real @ Ys + B_real @ Us

A, B = ou.alternating_minimization(Xs, Ys, Us, Zs, reg=np.array(1e-8), 
                                max_iter=100, tol=1e-10, symmetric_A=True)

print(np.allclose(T_real, A))
print(np.allclose(B_real, B))
# print(np.linalg.norm(A-A.T))


#### Test of alt min with parametric dependence

r = 10
b = 43
Nt = 27
Ns = 25
p = 4

Xs = np.eye(r)
Ys = np.random.randn(r,Nt,Ns)
Us = np.random.randn(b,Nt,Ns)
nus = np.random.randn(p,Ns)

T_fac = np.random.randn(r,r,p)
T_real = T_fac + T_fac.transpose(1,0,2)
B_real = np.random.randn(r,b,p)

Zs = np.einsum('ij,jkp,ps,kas->ias', Xs, T_real, nus, Ys) + np.einsum('ibp,ps,bas->ias', B_real, nus, Us)

# print(Zs.shape, Us.shape)

A, B = ou.alternating_minimization(Xs, Ys, Us, Zs, nus, reg=np.array(1e-8), 
                                max_iter=100, tol=1e-10, symmetric_A=True)

print(np.allclose(T_real, A))
print(np.allclose(B_real, B))

Converged in 33 iterations.
True
True
Converged in 48 iterations.
True
True
